In [1]:
#packages
import numpy
import pandas as pd
import os
from riotwatcher import LolWatcher, ApiError

In [2]:
#need to put this into a class
lol_watcher = LolWatcher('RGAPI-fc4c9639-8acc-42d1-bbe0-78fa6033e143') #api key

In [3]:
#####################
## Players 
#####################

def get_Player(region, name):
    
    valid_region_names = ['oc1', 'na1', 'eun1']
    if region in valid_region_names:
        player = lol_watcher.summoner.by_name(region, name.lower())
        return lol_watcher.league.by_summoner('oc1', player['id'])
    else:
        raise ValueError(f'region name is not a known region. Try one of {valid_region_names}')    

#####################
## Patches 
#####################
        
def get_CurrentPatchVersions(region):
    
    valid_region_names = ['oce', 'na1', 'eune']
    if region in valid_region_names:
        return lol_watcher.data_dragon.versions_for_region(region)
    else:
        raise ValueError(f'region name is not a known region. Try one of {valid_region_names}')   

def get_AllPatchVersions():
    
    return [i for i in lol_watcher.data_dragon.versions_all() if not i.startswith('lolpatch')]

#####################
## Champions
#####################

def get_Champions(patchversion = 'latest'):
    
    if patchversion == 'latest':
        champions = [[(patch,champ) for champ in lol_watcher.data_dragon.champions(patch)['data']] for patch in [get_AllPatchVersions()[0]]] #nested by patch        
    elif patchversion == 'all':
        champions = [[(patch,champ) for champ in lol_watcher.data_dragon.champions(patch)['data']] for patch in get_AllPatchVersions()] #nested by patch
    else:
        champions = [[(patch, champ) for champ in lol_watcher.data_dragon.champions(patch)['data']] for patch in patchversion] #nested by patch
        
    return pd.DataFrame([val for sublist in champions for val in sublist], columns = ['patchversion', 'champion']) #compress into single list

def get_ChampionStats(champions):
        data = pd.DataFrame.from_dict([{
                'patch': patchversion, 
                'champion': champion, 
                'championtype': lol_watcher.data_dragon.champions(patchversion)['data'][champion]['tags'],
                'spellresource': lol_watcher.data_dragon.champions(patchversion)['data'][champion]['partype'],
                **lol_watcher.data_dragon.champions(patchversion)['data'][champion]['info'],
                **lol_watcher.data_dragon.champions(patchversion)['data'][champion]['stats']} 
            for patchversion, champion in champions.values.tolist()
        ])
        
        #create championtype
        return pd.concat([data, pd.get_dummies(data['championtype'].apply(pd.Series).stack(), prefix = 'championtype').sum(level=0)], axis = 1).drop(columns=['championtype'])

#####################
## Champion Abilities
#####################    
#riot api doesnt publish this. can use the below sights to webscrape the abilies and version history.
#however the format isnt desirable
#https://leagueoflegends.fandom.com/wiki/Annie/LoL
#https://leagueoflegends.fandom.com/wiki/Annie/LoL/Patch_history

#####################
## Items
#####################    
    
#lol_watcher.data_dragon.items('12.22.1')

#####################
## Summoner Spells
#####################    

#lol_watcher.data_dragon.summoner_spells('12.22.1')

#####################
## Runes
#####################  

#lol_watcher.data_dragon.runes_reforged('12.22.1')

#####################
## Game/Match
##################### 

#lol_watcher.match.by_id()

In [4]:
champions = get_Champions()[0:3]
champions

,patchversion,champion
0,12.23.1,Aatrox
1,12.23.1,Ahri
2,12.23.1,Akali


In [189]:
data = get_ChampionStats(champions)
data

,patch,champion,spellresource,attack,defense,magic,difficulty,hp,hpperlevel,mp,...,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed,championtype_Assassin,championtype_Fighter,championtype_Mage,championtype_Tank
0,12.22.1,Aatrox,Blood Well,8,4,3,4,650,114,0,...,0,0,60,5.0,2.5,0.651,0,1,0,1
1,12.22.1,Ahri,Mana,3,4,8,5,570,96,418,...,0,0,53,3.0,2.0,0.668,1,0,1,0
2,12.22.1,Akali,Energy,5,3,8,7,570,119,200,...,0,0,62,3.3,3.2,0.625,1,0,0,0


In [54]:
lol_watcher.data_dragon.champions('12.22.1').keys()

dict_keys(['type', 'format', 'version', 'data'])

# Data

In [102]:
version = '12.22.1'
pd.DataFrame.from_dict({
                **lol_watcher.data_dragon.items(version)['data']
}).drop(index=['description', 'colloq', 'plaintext', 'into', 'image', 'from', 'depth',
       'inStore', 'effect', 'consumed', 'stacks', 'hideFromAll',
       'consumeOnFull', 'specialRecipe', 'requiredChampion', 'requiredAlly'])

,1001,1004,1006,1011,1018,1026,1027,1028,1029,1031,...,7022,7023,7024,7025,7026,7027,7028,7050,8001,8020
name,Boots,Faerie Charm,Rejuvenation Bead,Giant's Belt,Cloak of Agility,Blasting Wand,Sapphire Crystal,Ruby Crystal,Cloth Armor,Chain Vest,...,Seat of Command,Equinox,Caesura,Leviathan,The Unspoken Parasite,Primordial Dawn,Infinite Convergence,Gangplank Placeholder,Anathema's Chains,Abyssal Mask
gold,"{'base': 300, 'purchasable': True, 'total': 30...","{'base': 250, 'purchasable': True, 'total': 25...","{'base': 300, 'purchasable': True, 'total': 30...","{'base': 500, 'purchasable': True, 'total': 90...","{'base': 600, 'purchasable': True, 'total': 60...","{'base': 850, 'purchasable': True, 'total': 85...","{'base': 350, 'purchasable': True, 'total': 35...","{'base': 400, 'purchasable': True, 'total': 40...","{'base': 300, 'purchasable': True, 'total': 30...","{'base': 500, 'purchasable': True, 'total': 80...",...,"{'base': 0, 'purchasable': False, 'total': 250...","{'base': 0, 'purchasable': False, 'total': 250...","{'base': 0, 'purchasable': False, 'total': 280...","{'base': 0, 'purchasable': False, 'total': 320...","{'base': 0, 'purchasable': False, 'total': 300...","{'base': 0, 'purchasable': False, 'total': 300...","{'base': 0, 'purchasable': False, 'total': 320...","{'base': 0, 'purchasable': False, 'total': 0, ...","{'base': 800, 'purchasable': True, 'total': 25...","{'base': 650, 'purchasable': True, 'total': 30..."
tags,[Boots],[ManaRegen],[HealthRegen],[Health],[CriticalStrike],[SpellDamage],[Mana],[Health],[Armor],[Armor],...,"[Health, SpellDamage, ManaRegen, CooldownReduc...","[Health, SpellBlock, Armor, AbilityHaste]","[Health, SpellDamage, Mana, NonbootsMovement, ...","[Health, HealthRegen, CooldownReduction, Abili...","[Health, SpellBlock, Armor, CooldownReduction,...","[Health, SpellBlock, Armor, Aura, CooldownRedu...","[Health, HealthRegen, SpellDamage, Mana, ManaR...",[],"[Health, Active, CooldownReduction, AbilityHaste]","[Health, SpellBlock, HealthRegen, Mana, ManaRe..."
maps,"{'11': True, '12': True, '21': True, '22': False}","{'11': True, '12': True, '21': True, '22': False}","{'11': True, '12': True, '21': True, '22': False}","{'11': True, '12': True, '21': True, '22': False}","{'11': True, '12': True, '21': True, '22': False}","{'11': True, '12': True, '21': True, '22': False}","{'11': True, '12': True, '21': True, '22': False}","{'11': True, '12': True, '21': True, '22': False}","{'11': True, '12': True, '21': True, '22': False}","{'11': True, '12': True, '21': True, '22': False}",...,"{'11': True, '12': True, '21': True, '22': False}","{'11': True, '12': True, '21': True, '22': False}","{'11': True, '12': True, '21': True, '22': False}","{'11': True, '12': True, '21': True, '22': False}","{'11': True, '12': True, '21': True, '22': False}","{'11': True, '12': True, '21': True, '22': False}","{'11': True, '12': True, '21': True, '22': False}","{'11': True, '12': True, '21': True, '22': False}","{'11': True, '12': True, '21': True, '22': False}","{'11': True, '12': True, '21': True, '22': False}"
stats,{'FlatMovementSpeedMod': 25},{},{},{'FlatHPPoolMod': 350},{'FlatCritChanceMod': 0.15},{'FlatMagicDamageMod': 40},{'FlatMPPoolMod': 250},{'FlatHPPoolMod': 150},{'FlatArmorMod': 15},{'FlatArmorMod': 40},...,"{'FlatHPPoolMod': 300, 'FlatMagicDamageMod': 70}","{'FlatHPPoolMod': 400, 'FlatSpellBlockMod': 40...","{'FlatHPPoolMod': 350, 'FlatMPPoolMod': 800, '...",{'FlatHPPoolMod': 1050},"{'FlatHPPoolMod': 550, 'FlatSpellBlockMod': 40...","{'FlatHPPoolMod': 550, 'FlatSpellBlockMod': 40...","{'FlatHPPoolMod': 450, 'FlatMPPoolMod': 550, '...",{},{'FlatHPPoolMod': 650},"{'FlatHPPoolMod': 500, 'FlatSpellBlockMod': 40..."


In [103]:
lol_watcher.data_dragon.items(version)['data']['7050']

{'name': 'Gangplank Placeholder',
 'description': '',
 'colloq': '',
 'plaintext': 'New Gangplank interface coming soon!',
 'stacks': 0,
 'consumed': True,
 'consumeOnFull': True,
 'inStore': False,
 'requiredChampion': 'Gangplank',
 'hideFromAll': True,
 'image': {'full': '7050.png',
  'sprite': 'item2.png',
  'group': 'item',
  'x': 144,
  'y': 288,
  'w': 48,
  'h': 48},
 'gold': {'base': 0, 'purchasable': False, 'total': 0, 'sell': 0},
 'tags': [],
 'maps': {'11': True, '12': True, '21': True, '22': False},
 'stats': {}}